In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Установим стили для графиков
plt.style.use('default')
sns.set_palette("husl")

In [ ]:
def load_and_explore_data():
    # Загружаем основные данные
    dd_transactions = dd.read_parquet('transaction_fraud_data.parquet', engine='pyarrow')
    df_transactions = dd_transactions.compute()
    dd_currency = dd.read_parquet('historical_currency_exchange.parquet', engine='pyarrow')
    df_currency = dd_currency.compute()
    
    print("✅ Данные успешно загружены!")
    print(f"Транзакции: {df_transactions.shape}")
    print(f"Курсы валют: {df_currency.shape}")
    
    return df_transactions, df_currency
    
df, currency_df = load_and_explore_data()
# Преобразование timestamp к datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
def amount_usd(df):    
    ddf = dd.from_pandas(df, npartitions=4)
    currency_ddf = dd.from_pandas(currency_df, npartitions=1)
    avg_exchange_rates = currency_ddf.mean(numeric_only=True).compute()
    currency_to_usd = {currency: 1 / rate for currency, rate in avg_exchange_rates.items()}
    rates_df = dd.from_pandas(
        pd.DataFrame(list(currency_to_usd.items()), columns=["currency", "rate"]),
        npartitions=1
    )
    ddf = ddf.merge(rates_df, on="currency", how="left")
    ddf["rate"] = ddf["rate"].fillna(1)
    ddf["amount_usd"] = (ddf["amount"] * ddf["rate"])
    df = ddf.compute()
    return df

df = amount_usd(df)

### 1. Разведочный анализ данных (EDA)

#### Общая статистика

In [ ]:
def gen_stat(df):
    # Размер датасета
    total_transactions = len(df)
    unique_customers = df['customer_id'].nunique()
    unique_vendors = df['vendor'].nunique()
    unique_countries = df['country'].nunique()

    print(f"Размер датасета: {total_transactions:,} транзакций")
    print(f"Уникальных клиентов: {unique_customers:,}")
    print(f"Уникальных вендоров: {unique_vendors:,}")
    print(f"Уникальных стран: {unique_countries}")

    # Дисбаланс классов
    fraud_count = df['is_fraud'].sum()
    fraud_rate = df['is_fraud'].mean()
    legitimate_count = total_transactions - fraud_count

    print(f"\n🚨 ДИСБАЛАНС КЛАССОВ:")
    print(f"Легитимные операции: {legitimate_count:,} ({1-fraud_rate:.1%})")
    print(f"Мошеннические операции: {fraud_count:,} ({fraud_rate:.1%})")
    print(f"Соотношение легит:мошенн = {legitimate_count/fraud_count:.1f}:1")

    # Распределение по каналам
    print(f"\n📡 РАСПРЕДЕЛЕНИЕ ПО КАНАЛАМ:")
    channel_dist = df['channel'].value_counts()
    for channel, count in channel_dist.items():
        pct = count / total_transactions * 100
        print(f"  {channel}: {count:,} ({pct:.1f}%)")

    # Распределение по устройствам
    print(f"\n📱 РАСПРЕДЕЛЕНИЕ ПО УСТРОЙСТВАМ:")
    device_dist = df['device'].value_counts()
    for device, count in device_dist.head().items():
        pct = count / total_transactions * 100
        print(f"  {device}: {count:,} ({pct:.1f}%)")

    # Распределение по типам карт
    print(f"\n💳 РАСПРЕДЕЛЕНИЕ ПО ТИПАМ КАРТ:")
    card_dist = df['card_type'].value_counts()
    for card, count in card_dist.items():
        pct = count / total_transactions * 100
        print(f"  {card}: {count:,} ({pct:.1f}%)")

    # Количество и сумма транзакций по категориям вендора
    print(f"\n🏪 ТРАНЗАКЦИИ ПО КАТЕГОРИЯМ ВЕНДОРОВ:")
    vendor_summary = df.groupby('vendor_category').agg({
        'transaction_id': 'count',
        'amount': 'sum'
    }).round(2)
    vendor_summary.columns = ['Количество', 'Общая_сумма']
    vendor_summary = vendor_summary.sort_values('Количество', ascending=False)

    for category, row in vendor_summary.iterrows():
        pct = row['Количество'] / total_transactions * 100
        print(f"  {category}: {row['Количество']:,} транзакций ({pct:.1f}%), ${row['Общая_сумма']:,.0f}")

gen_stat(df)

#### Временные паттерны

In [ ]:
def time_pattern(df):
    # Добавляем временные признаки
    df['hour'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.day_name()
    df['date'] = df['timestamp'].dt.date

    # Доля мошенничества в выходные
    weekend_fraud = df.groupby('is_weekend')['is_fraud'].agg(['count', 'sum', 'mean'])
    weekend_fraud.columns = ['Всего_транзакций', 'Мошеннических', 'Доля_мошенничества']

    print(f"\n📅 МОШЕННИЧЕСТВО В ВЫХОДНЫЕ:")
    for is_weekend, row in weekend_fraud.iterrows():
        weekend_label = "Выходные" if is_weekend else "Будни"
        print(f"  {weekend_label}: {row['Мошеннических']}/{row['Всего_транзакций']} = {row['Доля_мошенничества']:.3f}")

    # Доля мошенничества ночью (с 0 до 6 утра)
    df['is_night'] = df['hour'].between(0, 6)
    night_fraud = df.groupby('is_night')['is_fraud'].agg(['count', 'sum', 'mean'])
    night_fraud.columns = ['Всего_транзакций', 'Мошеннических', 'Доля_мошенничества']

    print(f"\n🌙 МОШЕННИЧЕСТВО НОЧЬЮ (0-6 часов):")
    for is_night, row in night_fraud.iterrows():
        time_label = "Ночь" if is_night else "День"
        print(f"  {time_label}: {row['Мошеннических']}/{row['Всего_транзакций']} = {row['Доля_мошенничества']:.3f}")

    # Активность по часам
    hourly_activity = df.groupby('hour').agg({
        'transaction_id': 'count',
        'is_fraud': ['sum', 'mean']
    }).round(4)
    hourly_activity.columns = ['Всего_транзакций', 'Мошеннических', 'Доля_мошенничества']

    print(f"\n⏰ АКТИВНОСТЬ ПО ЧАСАМ (топ-5 по мошенничеству):")
    top_fraud_hours = hourly_activity.sort_values('Доля_мошенничества', ascending=False).head()
    for hour, row in top_fraud_hours.iterrows():
        print(f"  {hour:02d}:00 - {row['Мошеннических']}/{row['Всего_транзакций']} = {row['Доля_мошенничества']:.3f}")

    # Сезонность - количество транзакций по дням
    daily_stats = df.groupby('date').agg({
        'transaction_id': 'count',
        'is_fraud': 'sum',
        'amount': 'sum'
    }).round(2)
    daily_stats.columns = ['Транзакций', 'Мошеннических', 'Общая_сумма']

    print(f"\n📊 СТАТИСТИКА ПО ДНЯМ (первые 5 дней):")
    for date, row in daily_stats.head().iterrows():
        fraud_rate_day = row['Мошеннических'] / row['Транзакций'] if row['Транзакций'] > 0 else 0
        print(f"  {date}: {row['Транзакций']} транзакций, {row['Мошеннических']} мошеннических ({fraud_rate_day:.3f})")

    print(f"\nТренд транзакций: от {daily_stats['Транзакций'].iloc[0]} до {daily_stats['Транзакций'].iloc[-1]} в день")

time_pattern(df)

#### Георграфия

In [ ]:
def georgraphy(df):
    # Транзакции в своей стране vs за ее пределами
    outside_country_stats = df.groupby('is_outside_home_country')['is_fraud'].agg(['count', 'sum', 'mean'])
    outside_country_stats.columns = ['Всего_транзакций', 'Мошеннических', 'Доля_мошенничества']

    print(f"🗺️ ТРАНЗАКЦИИ ПО ЛОКАЦИИ:")
    for is_outside, row in outside_country_stats.iterrows():
        location = "За пределами родной страны" if is_outside else "В родной стране"
        pct_total = row['Всего_транзакций'] / len(df) * 100
        print(f"  {location}: {row['Мошеннических']}/{row['Всего_транзакций']} = {row['Доля_мошенничества']:.3f} ({pct_total:.1f}% всех транзакций)")

    # Регионы с наибольшей долей мошенничества
    fraud_by_country = df.groupby('country')['is_fraud'].agg(['count', 'sum', 'mean']).round(4)
    fraud_by_country.columns = ['Всего_транзакций', 'Мошеннических', 'Доля_мошенничества']
    fraud_by_country = fraud_by_country.sort_values('Доля_мошенничества', ascending=False)

    print(f"\n🚩 ТОП-5 СТРАН ПО ДОЛЕ МОШЕННИЧЕСТВА:")
    for country, row in fraud_by_country.head().iterrows():
        if row['Всего_транзакций'] >= 100:  # Только страны с достаточным количеством транзакций
            print(f"  {country}: {row['Мошеннических']}/{row['Всего_транзакций']} = {row['Доля_мошенничества']:.3f}")

    # Пересчет суммы транзакций в USD
    print(f"\n💱 КОНВЕРТАЦИЯ В ЕДИНУЮ ВАЛЮТУ (USD):")

    # Используем средние курсы валют
    currency_rates = currency_df.drop('date', axis=1).mean()
    print("Средние курсы валют относительно USD:")
    for currency, rate in currency_rates.items():
        if currency != 'USD':
            print(f"  1 USD = {rate:.3f} {currency}")

    # Статистика сумм в USD
    total_volume_usd = df['amount_usd'].sum()
    fraud_volume_usd = df[df['is_fraud']]['amount_usd'].sum()

    print(f"\n💰 ОБЪЕМЫ В USD:")
    print(f"Общий оборот: ${total_volume_usd:,.0f}")
    print(f"Оборот мошеннических операций: ${fraud_volume_usd:,.0f}")
    print(f"Доля мошеннического оборота: {fraud_volume_usd/total_volume_usd:.2%}")

    # Средние суммы по странам
    country_amounts = df.groupby('country')['amount_usd'].agg(['count', 'mean', 'median']).round(2)
    country_amounts.columns = ['Транзакций', 'Средняя_сумма_USD', 'Медианная_сумма_USD']
    country_amounts = country_amounts.sort_values('Средняя_сумма_USD', ascending=False)

    print(f"\n💵 СРЕДНИЕ СУММЫ ТРАНЗАКЦИЙ ПО СТРАНАМ:")
    for country, row in country_amounts.head().iterrows():
        print(f"  {country}: {row['Транзакций']} транзакций, средняя ${row['Средняя_сумма_USD']:.0f}, медианная ${row['Медианная_сумма_USD']:.0f}")
    
    # Средние и медианные суммы для разных типов карт
    card_amounts = df.groupby('card_type')['amount_usd'].agg(['count', 'mean', 'median', 'std']).round(2)
    card_amounts.columns = ['Транзакций', 'Средняя_USD', 'Медианная_USD', 'Стд_откл_USD']

    print(f"💳 СУММЫ ТРАНЗАКЦИЙ ПО ТИПАМ КАРТ:")
    for card_type, row in card_amounts.iterrows():
        cv = row['Стд_откл_USD'] / row['Средняя_USD'] if row['Средняя_USD'] > 0 else 0  # Коэффициент вариации
        print(f"  {card_type}: средняя ${row['Средняя_USD']:.0f}, медианная ${row['Медианная_USD']:.0f}, вариация {cv:.2f}")

    # Анализ больших отклонений
    print(f"\n📊 АНАЛИЗ БОЛЬШИХ ОТКЛОНЕНИЙ:")
    q95 = df['amount_usd'].quantile(0.95)
    q99 = df['amount_usd'].quantile(0.99)
    max_amount = df['amount_usd'].max()

    large_transactions = df[df['amount_usd'] > q95]
    large_fraud_rate = large_transactions['is_fraud'].mean()
    normal_fraud_rate = df[df['amount_usd'] <= q95]['is_fraud'].mean()

    print(f"95-й процентиль: ${q95:.0f}")
    print(f"99-й процентиль: ${q99:.0f}")
    print(f"Максимальная сумма: ${max_amount:.0f}")
    print(f"Доля мошенничества в крупных транзакциях (>95%): {large_fraud_rate:.3f}")
    print(f"Доля мошенничества в обычных транзакциях (≤95%): {normal_fraud_rate:.3f}")
    print(f"Lift крупных транзакций: {large_fraud_rate/normal_fraud_rate:.2f}x")

georgraphy(df)

#### Итоги

In [ ]:
def results(df):
    fraud_rate = df['is_fraud'].mean()
    outside_country_stats = df.groupby('is_outside_home_country')['is_fraud'].agg(['count', 'sum', 'mean'])
    outside_country_stats.columns = ['Всего_транзакций', 'Мошеннических', 'Доля_мошенничества']
    fraud_by_country = df.groupby('country')['is_fraud'].agg(['count', 'sum', 'mean']).round(4)
    fraud_by_country.columns = ['Всего_транзакций', 'Мошеннических', 'Доля_мошенничества']
    fraud_by_country = fraud_by_country.sort_values('Доля_мошенничества', ascending=False)
    q95 = df['amount_usd'].quantile(0.95)
    q99 = df['amount_usd'].quantile(0.99)
    max_amount = df['amount_usd'].max()
    large_transactions = df[df['amount_usd'] > q95]
    large_fraud_rate = large_transactions['is_fraud'].mean()
    normal_fraud_rate = df[df['amount_usd'] <= q95]['is_fraud'].mean()
    total_volume_usd = df['amount_usd'].sum()
    fraud_volume_usd = df[df['is_fraud']]['amount_usd'].sum()
    
    print("\n" + "="*60)
    print("📋 КЛЮЧЕВЫЕ ИНСАЙТЫ И ВЫВОДЫ")
    print("="*60)
    
    print("\n🎯 ОСНОВНЫЕ НАХОДКИ:")
    
    print("\n1. ДИСБАЛАНС КЛАССОВ:")
    print(f"   • {fraud_rate:.1%} мошеннических транзакций (соотношение 14.3:1)")
    print(f"   • Это умеренный дисбаланс, подходящий для ML-моделей")
    
    print("\n2. ГЕОГРАФИЧЕСКИЕ ПАТТЕРНЫ:")
    outside_lift = outside_country_stats.loc[True, 'Доля_мошенничества'] / outside_country_stats.loc[False, 'Доля_мошенничества']
    print(f"   • Транзакции за пределами родной страны имеют {outside_lift:.2f}x больший риск")
    print(f"   • Топ рискованные страны: {', '.join(fraud_by_country.head(3).index.tolist())}")
    
    print("\n3. ВРЕМЕННЫЕ АНОМАЛИИ:")
    print(f"   • Мошенничество практически одинаково в выходные и будни")
    print(f"   • Ночные транзакции менее рискованны (возможно, меньше активности)")
    print(f"   • Пиковые часы мошенничества: 17:00, 22:00-23:00")
    
    print("\n4. ФИНАНСОВЫЕ ПАТТЕРНЫ:")
    print(f"   • Крупные транзакции (>95% перцентиль) имеют {large_fraud_rate/normal_fraud_rate:.2f}x больший риск")
    print(f"   • Доля мошеннического оборота ({fraud_volume_usd/total_volume_usd:.1%}) выше доли транзакций")
    print(f"   • Типы карт показывают схожие паттерны мошенничества")
    
    print(f"\n✅ EDA ЗАВЕРШЕН")
    print(f"Проанализировано {len(df):,} транзакций с {df.columns.nunique()} признаками")
    print(f"Выявлено {len([col for col in df.columns if 'is_' in col])} бинарных индикаторов риска")

results(df)

#### Иллюстрации

In [ ]:
plt.figure(figsize=(5, 4))
sns.countplot(x="is_fraud", data=df, palette="Set2")
plt.title("Распределение мошеннических и немошеннических транзакций")
plt.xlabel("is_fraud")
plt.ylabel("Количество")
plt.show()

plt.figure(figsize=(8, 5))
sns.histplot(df["amount"], bins=50, kde=True)
plt.title("Распределение суммы транзакций")
plt.xlabel("Сумма")
plt.show()

num_df = df.select_dtypes(include=["float64", "int64", "bool"])
plt.figure(figsize=(10, 8))
sns.heatmap(num_df.corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Корреляция числовых признаков")
plt.show()

if "timestamp" in df.columns:
    df["date"] = pd.to_datetime(df["timestamp"]).dt.date
    daily_counts = df.groupby("date")["transaction_id"].count()
    plt.figure(figsize=(12, 5))
    daily_counts.plot()
    plt.title("Количество транзакций по дням")
    plt.ylabel("Количество")
    plt.xlabel("Дата")
    plt.show()

top_countries = df["country"].value_counts().head(10)
plt.figure(figsize=(10, 5))
sns.barplot(x=top_countries.index, y=top_countries.values, palette="viridis")
plt.title("Топ-10 стран по количеству транзакций")
plt.ylabel("Количество транзакций")
plt.xticks(rotation=45)
plt.show()

print("\nEDA завершён.")